### RandomForest | train(5), test(2) // Target: for문으로 24개 각각 모두 예측 // Input: 전체(seq,time빼고) // 

# 데이터

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score


%matplotlib inline

In [2]:
# Data Loading

train=pd.read_csv('/home/iotadmin/notebooks/vehicle/Train_set.csv',index_col=0) #Seq 빠짐: index로
test=pd.read_csv('/home/iotadmin/notebooks/vehicle/Test_set.csv',index_col=0)

# 1. RandomForest | *for문으로 돌려보자~~*

In [3]:
fm6andTime = ['FLFX', 'FLFY', 'FLFZ', 'FLMX', 'FLMY', 'FLMZ', 'FRFX', 'FRFY', 'FRFZ', 'FRMX', 'FRMY', 'FRMZ', 
         'RLFX', 'RLFY', 'RLFZ', 'RLMX', 'RLMY', 'RLMZ', 'RRFX', 'RRFY', 'RRFZ', 'RRMX', 'RRMY', 'RRMZ', 'Time']
print(len(fm6andTime))


X_train = train.drop(fm6andTime, axis=1)
X_test = test.drop(fm6andTime, axis=1)
print("X_train.shape: {}, X_test.shape: {}".format(X_train.shape, X_test.shape))


# 스케일링 
scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)
print(np.mean(X_train_scaled), np.std(X_train_scaled))
print(np.mean(X_test_scaled), np.std(X_test_scaled))
print(X_train.describe())
print(X_test.describe())

25
X_train.shape: (966564, 41), X_test.shape: (361173, 41)
-7.487833666824036e-16 0.9999999999999991
0.08857900648785903 1.1835688473947805
                FLSD         FLSARF         FLSBLF       FLLCABFX  \
count  966564.000000  966564.000000  966564.000000  966564.000000   
mean      372.833260     -19.864641     -11.486687      26.611334   
std         7.041958      40.311363      10.414369      60.232644   
min       311.987600    -361.093500     -96.690690    -992.371000   
25%       370.400500     -41.861090     -16.748580      -6.061694   
50%       373.310500     -26.627710     -12.586400      25.045530   
75%       376.306800      -1.807465      -7.139349      61.532100   
max       462.257400     355.837300      82.081670     848.257900   

            FLLCABFY          FLTBF           FRSD         FRSARF  \
count  966564.000000  966564.000000  966564.000000  966564.000000   
mean       77.449077     -78.645498     343.296793     -26.843616   
std        42.948361      19.69

In [4]:
fm6 = ['FLFX', 'FLFY', 'FLFZ', 'FLMX', 'FLMY', 'FLMZ', 'FRFX', 'FRFY', 'FRFZ', 'FRMX', 'FRMY', 'FRMZ', 
       'RLFX', 'RLFY', 'RLFZ', 'RLMX', 'RLMY', 'RLMZ', 'RRFX', 'RRFY', 'RRFZ', 'RRMX', 'RRMY', 'RRMZ']



for case in fm6:
    
    y_train = train[case]
    y_test = test[case]
    print("y_train.shape: {}, y_test.shape: {}".format(y_train.shape, y_test.shape))
    
    
    # model 생성
    RF = RandomForestRegressor(n_estimators=100, n_jobs=-1, random_state=42)
    print(RF)

    
    # fit
    RF.fit(X_train_scaled, y_train)
    print(RF)


    # predict
    predictions = RF.predict(X_test_scaled)
    print(predictions.shape)
    
    
    # MSE | mean squared error 
    mse = mean_squared_error(y_test, predictions)

    
    # RMSE
    rmse = np.sqrt(mse)


    # MAE | mean absloute error
    mae = mean_absolute_error(y_test, predictions)


    # R2 / R square
    r2 = r2_score(y_test, predictions)

    
    # PearsonCorrelationSquare
    predict_data = pd.DataFrame(data = [y_test.values.reshape(-1), predictions])
    predict_data = predict_data.transpose()
    pc2 = predict_data.corr(method='pearson') ** 2
    
    
    print("MSE: ", mse)
    print("RMSE: ", rmse)
    print("MAE: ", mae)
    print("R^2: ", r2)
    print("PearsonCorrelation^2: ", pc2[1][0])

    
    # predict_data를 csv로 저장
    col = ["y_test", "predictions"]
    predict_data.columns = col
    predict_data.to_csv("/home/iotadmin/notebooks/RandomForest_FINAL/RandomForest_{}.csv".format(case))

    
    print("___________________________finished_{}".format(case))


y_train.shape: (966564,), y_test.shape: (361173,)
RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
           oob_score=False, random_state=42, verbose=0, warm_start=False)
RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
           oob_score=False, random_state=42, verbose=0, warm_start=False)
(361173,)
MSE:  238781.52153203473
RMSE:  488.652761715346
MAE:  154.54195822928594
R^2:  -17.54887453960971
PearsonCorrelation^2:  0.05115750886195219
___________________________finis

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
           oob_score=False, random_state=42, verbose=0, warm_start=False)
(361173,)
MSE:  85680.64268082057
RMSE:  292.7125598275902
MAE:  233.73522074478183
R^2:  0.3034685210171496
PearsonCorrelation^2:  0.5074763254589072
___________________________finished_RRFZ
y_train.shape: (966564,), y_test.shape: (361173,)
RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
           oob_score=False, random_state=42, verbose=0, warm_start

In [8]:
scoring = pd.concat([pd.Series(["MSE", "RMSE", "MAE", "PearsonCorrelation^2"]), pd.Series([mse, rmse, mae, pc2[1][0]])], axis=1)
scoring.columns = []

scoring


,0,1
0,MSE,1025.421253
1,RMSE,32.022199
2,MAE,19.103353
3,PearsonCorrelation^2,0.113252
